# André Fonseca
# Trabalho de conclusão do Data Science - Awari
# Análise derivada do LISH-MoA
## https://www.kaggle.com/c/lish-moa/overview

# 1. Import modules

In [1]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 3.7MB/s 


In [2]:
# Basic
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

# Pre-processing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import IterativeStratification

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import LabelPowerset
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.adapt import MLkNN
from skmultilearn.ensemble import LabelSpacePartitioningClassifier

# Optimization
from sklearn.model_selection import GridSearchCV

# Evaluation
from sklearn.metrics import hamming_loss
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score

#### Only if you are using Google Colab

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/My Drive/Awari

/content/drive/My Drive/Awari


# 2. Loading dataset

In [5]:
# Loading X data
train_features = pd.read_csv("./data/train_features.csv")
test_features = pd.read_csv("./data/test_features.csv")

In [6]:
# Loading Y data
train_targets_sc = pd.read_csv("./data/train_targets_scored.csv")
train_targets_ns = pd.read_csv("./data/train_targets_nonscored.csv")

In [7]:
sample_submission = pd.read_csv("./data/sample_submission.csv")

## 2.1. Preparing dataset to Machine Learning

In [8]:
X = train_features.drop('sig_id', axis = 1)
X = pd.get_dummies(X)

In [9]:
y = train_targets_sc.drop('sig_id', axis = 1)

## 2.2. Building a validation test

In [10]:
stratifier = IterativeStratification(n_splits=2, order=2, sample_distribution_per_fold=[0.25, 0.75])
train_indexes, test_indexes = next(stratifier.split(X, y))

X_train, y_train = X.loc[train_indexes], y.loc[train_indexes]
X_val, y_val = X.loc[test_indexes], y.loc[test_indexes]

## 2.3. Loading pre-calculated models

In [108]:
power = pickle.load(open('./data/models/power_model.pkl', 'rb'))

In [ ]:
br = pickle.load(open('./data/models/binary_forest_model.pkl', 'rb'))

In [ ]:
knn = pickle.load(open('./data/models/mlknn_model.pkl', 'rb'))

In [ ]:
knn_grid = pickle.load(open('./data/models/knn_cv_model.pkl', 'rb'))

In [ ]:
lab_space = pickle.load(open('./data/models/lab_space.pkl', 'rb'))

In [ ]:
lab_grid = pickle.load(open('./data/models/lab_cv_model.pkl', 'rb'))

# 3. Multi-labels models

### Problem Transformation approaches
* One-vs-Rest - Logistic Regression
* LabelPowerSet - Logistic Regression
* BinaryRelevance - Random Forest

### Algorithm Adaptation approaches
* multi-label adapted kNN
* multi-label adapted kNN + Cross-Validation

### Ensembles of Classifiers
* LabelSpacePartitioningClassifier
* LabelSpacePartitioningClassifier + Cross-Validation

## 3.1. One-vs-Rest

In [18]:
moa_category, moa_names, moa_accuracy = y_train.columns, [], []

In [14]:
log_pipeline = Pipeline([('classifier', OneVsRestClassifier(
    LogisticRegression(solver = 'sag'), n_jobs = -1)), ])

CPU times: user 138 µs, sys: 23 µs, total: 161 µs
Wall time: 164 µs


In [19]:
%%time

for category in moa_category:
  try:
    log_pipeline.fit(X_train, y_train[category])
  except:
    print("Error!")
  
  y_pred = log_pipeline.predict(X_val)
  score = accuracy_score(y_val[category], y_pred)

  moa_accuracy.append(score)
  moa_names.append(category)

CPU times: user 18.7 s, sys: 33.3 s, total: 52.1 s
Wall time: 51min 32s


In [ ]:
log_one_vs_rest = pd.DataFrame({'MoA': moa_names, 'accuracy': moa_accuracy})

In [ ]:
log_one_vs_rest.head(10)

In [ ]:
log_one_vs_rest.to_csv("./data/models/log_one_vs_rest.csv", index = False)

## 3.2. LabelPowerSet

In [102]:
power = LabelPowerset(LogisticRegression(), 
                      require_dense = [False, True])

In [ ]:
%%time
power.fit(X_train, y_train)

In [46]:
y_pred = power.predict(X_val)

In [104]:
y_prob = power.predict_proba(X_val)

In [105]:
y_prob_array = y_prob.toarray()

In [ ]:
accuracy_score = accuracy_score(y_val.values, y_pred)

In [16]:
print(f'Accuracy score: {accuracy_score}')

Accuracy score: 0.47195162915686933


In [17]:
f1_score = f1_score(y_val.values, y_pred, average = 'micro')

In [18]:
print(f'F1-score: {accuracy_score}')

F1-score: 0.47195162915686933


In [106]:
logging_score = log_loss(y_val.values, y_prob_array)

In [20]:
print(f'Log-loss score: {logging_score}')

Log-loss score: 2.3457120387845443


In [21]:
hamming_loss = hamming_loss(y_val.values, y_pred)

In [22]:
print(f'Hamming-loss score: {hamming_loss}')

Hamming-loss score: 0.003414527559183514


In [33]:
with open('./data/models/power_model.pkl','wb') as outfile:
  pickle.dump(power, outfile)

## 3.3. BinaryRelevance

In [34]:
br = BinaryRelevance(
    classifier = RandomForestClassifier(),
    require_dense = [False, True]
)

In [35]:
%%time
br.fit(X_train, y_train)

CPU times: user 3h 17min 49s, sys: 2.84 s, total: 3h 17min 51s
Wall time: 3h 18min 59s


BinaryRelevance(classifier=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                  class_weight=None,
                                                  criterion='gini',
                                                  max_depth=None,
                                                  max_features='auto',
                                                  max_leaf_nodes=None,
                                                  max_samples=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100, n_jobs=None,
                                                  oo

In [36]:
y_pred = br.predict(X_val)

In [50]:
y_prob = br.predict_proba(X_val)

In [51]:
y_prob_array = y_prob.toarray()

In [ ]:
accuracy_score = accuracy_score(y_val.values, y_pred)

In [ ]:
print(f'Accuracy score: {accuracy_score}')

In [ ]:
f1_score = f1_score(y_val.values, y_pred, average = 'micro')

In [54]:
print(f'F1-score: {accuracy_score}')

F1-score: 0.47195162915686933


In [59]:
logging_score = log_loss(y_val.values, y_prob_array)

In [60]:
print(f'Log-loss score: {logging_score}')

Log-loss score: 7.001000542703964


In [ ]:
hamming_loss = hamming_loss(y_val.values, y_pred)

In [58]:
print(f'Hamming-loss score: {hamming_loss}')

Hamming-loss score: 0.003414527559183514


In [ ]:
with open('./data/models/binary_forest_model.pkl','wb') as outfile:
  pickle.dump(br, outfile)

## 3.4. multi-label adapted kNN

In [85]:
knn = MLkNN(k = 5)

In [86]:
%%time
knn.fit(X_train.values, y_train.values)

CPU times: user 9min 35s, sys: 157 ms, total: 9min 35s
Wall time: 9min 35s


MLkNN(ignore_first_neighbours=0, k=5, s=1.0)

In [87]:
y_pred = knn.predict(X_val)

In [ ]:
y_prob = knn.predict_proba(X_val)

In [89]:
y_prob_array = y_prob.toarray()

In [ ]:
accuracy_score = accuracy_score(y_val.values, y_pred)

In [ ]:
print(f'Accuracy score: {accuracy_score}')

In [ ]:
f1_score = f1_score(y_val.values, y_pred, average = 'micro')

In [ ]:
print(f'F1-score: {accuracy_score}')

In [90]:
logging_score = log_loss(y_val.values, y_prob_array)

In [91]:
print(f'Log-loss score: {logging_score}')

Log-loss score: 8.653149769241583


In [ ]:
hamming_loss = hamming_loss(y_val.values, y_pred)

In [ ]:
print(f'Hamming-loss score: {hamming_loss}')

In [88]:
with open('./data/models/mlknn_model.pkl','wb') as outfile:
  pickle.dump(knn, outfile)

## 3.5. multi-label adapted kNN + Cross-Validation

In [ ]:
parameters = {'k': range(8, 10), 's': [0.5, 1.0]}
knn_grid = GridSearchCV(MLkNN(), parameters, scoring = 'f1_micro')

In [ ]:
%%time
knn_grid.fit(X_train, y_train.values)

In [61]:
print(f'Best parameters: {knn_grid.best_params_}.\nBest score: {knn_grid.best_score_}')

Best parameters: {'k': 9, 's': 1.0}.
Best score: 0.28548788186446944


In [62]:
y_pred = knn_grid.predict(X_val)

In [63]:
y_prob = knn_grid.predict_proba(X_val)

In [64]:
y_prob_array = y_prob.toarray()

In [ ]:
accuracy_score = accuracy_score(y_val.values, y_pred)

In [ ]:
print(f'Accuracy score: {accuracy_score}')

In [ ]:
f1_score = f1_score(y_val.values, y_pred, average = 'micro')

In [67]:
print(f'F1-score: {accuracy_score}')

F1-score: 0.47195162915686933


In [68]:
logging_score = log_loss(y_val.values, y_prob_array)

In [69]:
print(f'Log-loss score: {logging_score}')

Log-loss score: 3.0512107029335276


In [ ]:
hamming_loss = hamming_loss(y_val.values, y_pred)

In [71]:
print(f'Hamming-loss score: {hamming_loss}')

Hamming-loss score: 0.003414527559183514


In [ ]:
with open('./data/models/knn_cv_model.pkl','wb') as outfile:
  pickle.dump(knn_grid, outfile)

## 3.6. LabelSpacePartitioningClassifier

In [92]:
# Load Clusterer object
clusterer = pickle.load(open('./data/clusterer.pkl', 'rb'))

In [93]:
lab_space = LabelSpacePartitioningClassifier(
    classifier = BinaryRelevance(
    classifier = RandomForestClassifier(),
    require_dense = [False, True]
    ),
    clusterer  = clusterer
)

In [94]:
%%time
lab_space.fit(X_train,y_train)

CPU times: user 3h 13min 46s, sys: 2.83 s, total: 3h 13min 49s
Wall time: 3h 13min 57s


LabelSpacePartitioningClassifier(classifier=BinaryRelevance(classifier=RandomForestClassifier(bootstrap=True,
                                                                                              ccp_alpha=0.0,
                                                                                              class_weight=None,
                                                                                              criterion='gini',
                                                                                              max_depth=None,
                                                                                              max_features='auto',
                                                                                              max_leaf_nodes=None,
                                                                                              max_samples=None,
                                                                                              min_impuri

In [96]:
y_pred = lab_space.predict(X_val)

In [97]:
y_prob = lab_space.predict_proba(X_val)

In [98]:
y_prob_array = y_prob.toarray()

In [ ]:
accuracy_score = accuracy_score(y_val.values, y_pred)

In [ ]:
print(f'Accuracy score: {accuracy_score}')

In [ ]:
f1_score = f1_score(y_val.values, y_pred, average = 'micro')

In [ ]:
print(f'F1-score: {accuracy_score}')

In [100]:
logging_score = log_loss(y_val.values, y_prob_array)

In [101]:
print(f'Log-loss score: {logging_score}')

Log-loss score: 10.375247354136405


In [ ]:
hamming_loss = hamming_loss(y_val.values, y_pred)

In [ ]:
print(f'Hamming-loss score: {hamming_loss}')

In [95]:
with open('./data/models/lab_space.pkl','wb') as outfile:
  pickle.dump(lab_space, outfile)

## 3.7. LabelSpacePartitioningClassifier + Cross-Validation

In [ ]:
parameters = {
    'classifier': [BinaryRelevance()],
    'classifier__classifier': [RandomForestClassifier()],
    'classifier__classifier__n_estimators': [10, 20, 50],
    'clusterer' : [
        clusterer
    ]
}

In [ ]:
lab_grid = GridSearchCV(LabelSpacePartitioningClassifier(), parameters, scoring = 'f1_micro')

In [ ]:
%%time
lab_grid.fit(X_train, y_train)

In [ ]:
print(f'Best parameters: {lab_grid.best_params_}.\nBest score: {lab_grid.best_score_}')

In [75]:
y_pred = lab_grid.predict(X_val)

In [76]:
y_prob = lab_grid.predict_proba(X_val)

In [77]:
y_prob_array = y_prob.toarray()

In [ ]:
accuracy_score = accuracy_score(y_val.values, y_pred)

In [ ]:
print(f'Accuracy score: {accuracy_score}')

In [ ]:
f1_score = f1_score(y_val.values, y_pred, average = 'micro')

In [ ]:
print(f'F1-score: {accuracy_score}')

In [81]:
logging_score = log_loss(y_val.values, y_prob_array)

In [82]:
print(f'Log-loss score: {logging_score}')

Log-loss score: 8.653149769241583


In [ ]:
hamming_loss = hamming_loss(y_val.values, y_pred)

In [84]:
print(f'Hamming-loss score: {hamming_loss}')

Hamming-loss score: 0.003414527559183514


In [ ]:
with open('./data/models/lab_cv_model.pkl','wb') as outfile:
  pickle.dump(lab_grid, outfile)

# 4. References

In [ ]:
# https://skml.readthedocs.io/en/latest/auto_examples/example_lp.html
# https://xang1234.github.io/multi-label/
# https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff
# http://scikit.ml/modelselection.html
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.multilabel_confusion_matrix.html